In [1]:
!pip install kloppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 993.1 kB/s eta 0:00:00


In [2]:
from kloppy import datasets, wyscout

In [3]:
manu_westham = datasets.load('wyscout', match_id = 2499724)

In [4]:
manu_westham.to_pandas()

,event_id,event_type,result,success,period_id,timestamp,end_timestamp,ball_state,ball_owning_team,team_id,...,coordinates_y,end_coordinates_x,end_coordinates_y,receiver_player_id,is_counter_attack,pass_type,set_piece_type,body_part_type,goalkeeper_action_type,card_type
0,178494347,PASS,COMPLETE,True,1,5.546151,None,None,None,1611,...,0.52,0.40,0.44,NaN,False,SIMPLE_PASS,NaN,NaN,NaN,NaN
1,178494348,PASS,COMPLETE,True,1,6.708916,None,None,None,1611,...,0.44,0.33,0.75,NaN,False,SIMPLE_PASS,NaN,NaN,NaN,NaN
2,178494349,PASS,COMPLETE,True,1,9.612072,None,None,None,1611,...,0.75,0.42,0.76,NaN,False,SIMPLE_PASS,NaN,NaN,NaN,NaN
3,178494350,PASS,COMPLETE,True,1,10.718877,None,None,None,1611,...,0.76,0.28,0.36,NaN,False,SIMPLE_PASS,NaN,NaN,NaN,NaN
4,178494351,PASS,COMPLETE,True,1,12.728071,None,None,None,1611,...,0.36,0.34,0.12,NaN,False,SIMPLE_PASS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,178496018,RECOVERY,None,None,2,2862.313235,None,None,None,1633,...,0.35,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
1642,178496019,BALL_OUT,None,None,2,2866.414500,None,None,None,1633,...,0.00,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
1643,178495982,PASS,COMPLETE,True,2,2879.021252,None,None,None,1611,...,1.00,0.93,0.91,NaN,False,HAND_PASS,THROW_IN,NaN,NaN,NaN
1644,178496020,TAKE_ON,None,None,2,2883.667683,None,None,None,1633,...,0.09,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN


In [5]:
import json
import pandas as pd
import numpy as np

In [6]:
json_path = '../input/wyscoutdata/matches_England.json'
with open(json_path) as f:
    data = json.load(f)
    matches = pd.json_normalize(data, sep = "_")

In [7]:
matches['gameweek']

0      38
1      38
2      38
3      38
4      38
       ..
375     1
376     1
377     1
378     1
379     1
Name: gameweek, Length: 380, dtype: int64

In [8]:
matches.label.iloc[0].split(' - ')[1].split(', ')

['AFC Bournemouth', '1']

In [9]:
def split_label(string):
    home, away_home_score, away_score = string.split(' - ')
    away, home_score = away_home_score.split(', ')
    return home, away, home_score, away_score

In [10]:
matches['home_team'],matches['away_team'], matches['home_score'], matches['away_score'] = zip(*matches['label'].map(split_label))

In [11]:
scores = matches[['home_team', 'away_team', 'home_score', 'away_score', 'gameweek']]

In [12]:
teams = matches['home_team'].unique()
teams

array(['Burnley', 'Crystal Palace', 'Huddersfield Town', 'Liverpool',
       'Manchester United', 'Newcastle United', 'Southampton',
       'Swansea City', 'Tottenham Hotspur', 'West Ham United',
       'Manchester City', 'Leicester City', 'Chelsea', 'Arsenal',
       'Everton', 'AFC Bournemouth', 'Watford', 'West Bromwich Albion',
       'Stoke City', 'Brighton & Hove Albion'], dtype=object)

In [13]:
scores['home_score'] = scores['home_score'].astype(np.int)
scores['away_score'] = scores['away_score'].astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython 

In [14]:
table = {}
for team in teams:
    table[team] = {'goals': 0, 'goals_against': 0, 'points': 0, 'wins': 0, 'draws': 0, 'losses': 0}
for index, game in scores.iterrows():
    home_team = game['home_team']
    away_team = game['away_team']
    away_score = game['away_score']
    home_score = game['home_score']
    #update scores
    table[home_team]['goals'] += home_score
    table[home_team]['goals_against'] += away_score
    table[away_team]['goals'] += away_score
    table[away_team]['goals_against'] += home_score
    #update wins
    if home_score > away_score:
        table[home_team]['wins'] += 1
        table[away_team]['losses'] += 1 
    elif home_score == away_score:
        table[home_team]['draws'] += 1
        table[away_team]['draws'] += 1
    else:
        table[home_team]['losses'] += 1
        table[away_team]['wins'] += 1

In [15]:
table = pd.DataFrame().from_dict(table, orient='index')

In [16]:
table['points'] = table['wins']*3 + table['draws']

In [17]:
table.sort_values(by = 'points')

,goals,goals_against,points,wins,draws,losses
West Bromwich Albion,31,56,31,6,13,19
Stoke City,35,68,33,7,12,19
Swansea City,28,56,33,8,9,21
Southampton,37,56,36,7,15,16
Huddersfield Town,28,58,37,9,10,19
Brighton & Hove Albion,34,54,40,9,13,16
Watford,44,64,41,11,8,19
West Ham United,48,68,42,10,12,16
AFC Bournemouth,45,61,44,11,11,16
Crystal Palace,45,55,44,11,11,16


In [18]:
table.to_csv("epl_table.csv")